This notebook outputs a csv file containing the **Variation** table. 
This table is obtained from the Churn_usersProfile.txt file. It contains, for each of the demographics (combinations of Origin County, Gender and Age), a value for the **variation in travellers between the first and second time periods** (in relative and absolute terms).  
If the variation values are negative for a row, that means there was churn for the specific demographic.  
Note that there are a number of demographics for which only the value in travellers for the first or second was captured. These cases are left out from this 'Variation' table.

In [1]:
import pandas as pd
import fiona
from IPython.display import display

Importing the user profiles Table

In [2]:
Churn_UsersProfile = pd.read_csv('https://wdl-data.fra1.digitaloceanspaces.com/pse/Churn_UsersProfile.txt',
                                 encoding = "ISO-8859-1",
                                sep='|')

Remove rows of table for which only a single number of travellers was recorded (eg. there is only a value for 'Sep-19 to Feb-20', but not for' Sep-20 to Jan-21')

In [3]:
profiles_gb = Churn_UsersProfile.groupby(['County_of_Origin','GenderDescription', 'AgeClassDescription']).size().to_frame()

In [4]:
single_period = profiles_gb[profiles_gb.values==1].reset_index()

In [5]:
two_periods = Churn_UsersProfile.merge(single_period, how='outer', indicator=True).query('_merge != "both"').drop(['_merge', 0], 1)

Create two separate dataframes for values for the first and second time periods

In [6]:
df_first = two_periods[two_periods.Period == 'Sep-19 to Feb-20'].drop('Period', axis=1).rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_first'})
df_second = two_periods[two_periods.Period == 'Sep-20 to Jan-21'].drop('Period', axis=1).rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_second'})

Merge the two dataframes, so that we have in a single row the number of travellers for the first and second time periods

In [7]:
variation_df = df_first.merge(df_second, how='outer', indicator=True).drop('_merge', 1)

Calculate relative and absolute variation values

In [8]:
variation_df['Variation_abs'] = variation_df.Average_BusUsers_per_Day_second - variation_df.Average_BusUsers_per_Day_first 
variation_df['Variation_rel'] = (variation_df.Average_BusUsers_per_Day_second - variation_df.Average_BusUsers_per_Day_first ) / variation_df.Average_BusUsers_per_Day_first
#variation_df = variation_df.drop(['Average_BusUsers_per_Day_first', 'Average_BusUsers_per_Day_second'], 1)

Output to csv

In [11]:
variation_df.to_csv('intermediate-data/variation.csv', index=False)